# Get Dataset

In [ ]:
# Get roboflow data for pickleball court keypoints detection training
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="K5y7hyQBmDKQ82NzgM5Q")
project = rf.workspace("pickleball-ball-detection").project("pickleball-court-keypoints-syncz")
version = project.version(6)
dataset = version.download("coco")

# Start Code

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [ ]:
class keyPointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astytpe(np.float32)

        # Adjust keypoints
        kps[::2] *= (224 / w) # Adjust x coordinates
        kps[1::2] *= (224 / h) # Adjust y coordinates

        return img, kps
    

In [ ]:
# Using RoboFlow JSON COCO format
# Adjust file paths
from pathlib import Path

root = Path(dataset.location)
inner = root / "Pickleball-1" if (root / "Pickleball-1").exists() else root  # handle your earlier nesting

train_images = inner / "train"
val_images   = inner / ("valid" if (inner/"valid").exists() else "val")

# common Roboflow layout:
train_ann = train_images / "_annotations.coco.json"
val_ann   = val_images   / "_annotations.coco.json"

# fallbacks if your export uses a central annotations dir or different names
if not train_ann.exists():
    alt = inner / "annotations" / "data_train.json"
    if alt.exists():
        train_ann = alt
if not val_ann.exists():
    alt = inner / "annotations" / "data_val.json"
    if alt.exists():
        val_ann = alt

print(train_images, train_ann)
print(val_images, val_ann)


In [ ]:
# Obtain datasets
train_dataset = keyPointsDataset(str(train_images), str(train_ann))
val_dataset   = keyPointsDataset(str(val_images), str(val_ann))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [ ]:
model = models.resnet50(pretrained=True) 